# Scottish ZEV Mandate Carbon Reduction Analysis

In [1]:

import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import re
import requests
import os
import numpy as np
tqdm.pandas()
%load_ext google.cloud.bigquery

## Step 1: Pre-process data

### 1.1 Fuel Consumption Data
The Department for Business, Energy Security and Industrial Strategy (as was) publishes sub-national fuel consumption data, from which we can extract data for Scotland. Unfortunately they publish one year on one tab of the document, meaning we need to write a little script to extract the data and put it in a better format.

Input: 'sub-national-road-transport-fuel-consumption-statistics-2005-20**.ods' from inside the `data` folder, available at https://www.gov.uk/government/collections/road-transport-consumption-at-regional-and-local-level

Output: a CSV file with cleaned up fuel consumption data, where each row is a year, so that we can see the trend over time. The csv `scotland_yearly_fuel.csv` is saved in the data folder.

In [2]:
if not os.path.isfile('data/scotland_yearly_fuel.csv'):

    years = list(range(2005, 2021))
    years = [str(x) for x in years]

    fl = []

    for year in tqdm(years):
        df = pd.read_excel('data/sub-national-road-transport-fuel-consumption-statistics-2005-2020.ods', header=3, sheet_name=year)
        df = df[df['Local Authority Code'] == 'S12000033']
        df['year'] = int(year)
        fl.append(df)

    fc = pd.concat(fl)

    fc.set_index('year', inplace=True)

    fc.index = pd.to_datetime(fc.index, format='%Y')

    fc.to_csv('data/scotland_yearly_fuel.csv')
else:
    fc = pd.read_csv('data/scotland_yearly_fuel.csv', index_col=0)
    fc.index = pd.to_datetime(fc.index, format='%Y')

100%|██████████| 16/16 [05:13<00:00, 19.59s/it]


### 1.2 Vehicle Fleet Composition Data
We want to know how many cars are on the road in Scotland. DfT publishes two tables which cover this, from their Vehicle Licensing Statistics data series (here: https://www.gov.uk/government/statistical-data-sets/vehicle-licensing-statistics-data-tables).

The two tables are `VEH0105` and `VEH0142` covering the number of petrol, diesel and other fossil fuelled vehicles, and different kinds of low emissions vehicles respectively. Because these are in two different sheets, we need a script (below) that will clean them up and make them more usable. It's very slow, unfortunately.

Inputs: 'veh0105.ods' and 'veh0142' from the data folder. They are saved into one CSV called scotland_vehicle_parc.csv

In [3]:
#crazy function to rationalise fueltypes
def ft_cleaner_dft(x):
    x = x.replace('Battery electric', 'Pure Electric')
    x = x.replace('Other fuels', 'Other').replace('Other fuel types', 'Other')
    x = x.replace('Plug-in hybrid electric (diesel)', 'Diesel')
    x = x.replace('Range extended electric', 'Pure Electric')
    x = x.replace('Plug-in hybrid electric (petrol)', 'Petrol')
    x = x.replace('Hybrid electric (petrol)', 'Petrol')
    x = x.replace('Hybrid electric (diesel)', 'Diesel')
    x = x.replace('Fuel cell electric', 'Pure Electric')
    x = x.replace('Gas', 'Other')
    return x

In [4]:
if not os.path.isfile('data/scotand_vehicle_parc.csv'):

    # This deals with VEH0105. We're basically taking the ODS format document, and then dropping irrelevant bits of the table. Unfortunately as DfT like to change the way they format their spreadsheets, this will likely break in the future or need amendment to keep it working OK. 

    veh0105 = pd.read_excel('data/veh0105.ods', sheet_name='VEH0105', header=4)
    veh0105.drop(columns=['Units', 'ONS Geography [note 6]'], inplace=True)
    veh0105 = veh0105[veh0105.BodyType != 'Total']
    veh0105 = veh0105[veh0105['Fuel [note 2]'] != 'Total']
    veh0105 = veh0105[veh0105['Keepership [note 3]'] != 'Total']
    veh0105 = veh0105[veh0105['ONS Code [note 6]'] == 'S12000033']
    veh0105.drop(columns=['ONS Sort [note 6]', 'ONS Code [note 6]'], inplace=True)
    veh0105.rename(columns={'Fuel [note 2]': 'Fuel', 'Keepership [note 3]': 'Keepership'}, inplace=True)
    veh0105 = veh0105.melt(id_vars=['BodyType', 'Fuel', 'Keepership'])
    veh0105['value'] = veh0105.value.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    veh0105['variable'] = pd.PeriodIndex(veh0105['variable'].apply(lambda x: x.replace(' ', '-')), freq='Q').to_timestamp()
    veh0105['value'] = veh0105['value'].apply(lambda x: x*1000)
    
    # Now import VEH0142
    veh0142 = pd.read_excel('data/veh0142.ods', sheet_name='VEH0142', header=4)
    veh0142.drop(columns=['Units', 'ONS Geography [note 6]'], inplace=True)
    veh0142 = veh0142[veh0142['Fuel'] != 'Total']
    veh0142 = veh0142[veh0142['BodyType'] != 'Total']
    veh0142 = veh0142[veh0142['Keepership [note 3]'] != 'Total']
    veh0142 = veh0142[veh0142['ONS Code [note 6]'] == 'S12000033']
    veh0142.drop(columns=['ONS Sort [note 6]', 'ONS Code [note 6]'], inplace=True)
    veh0142.rename(columns={'Fuel [note 2]': 'Fuel', 'Keepership [note 3]': 'Keepership'}, inplace=True)
    veh0142 = veh0142.melt(id_vars=['Fuel', 'Keepership', 'BodyType'])
    veh0142['value'] = veh0142.value.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    veh0142['variable'] = pd.PeriodIndex(veh0142['variable'].apply(lambda x: x.replace(' ', '-')), freq='Q').to_timestamp()
    
    pc = pd.concat([veh0142, veh0105])
    pc['variable'] = pc.variable.apply(pd.to_datetime)
    pc = pc.groupby(['variable', 'BodyType', 'Fuel']).sum().reset_index(['BodyType', 'Fuel'])
    pc['Fuel'] = pc.Fuel.apply(ft_cleaner_dft)
    pc = pc.reset_index().groupby(['variable', 'BodyType', 'Fuel']).sum().reset_index(['BodyType', 'Fuel'])
    
    pc.to_csv('data/scotand_vehicle_parc.csv')
    
else:
    pc = pd.read_csv('data/scotand_vehicle_parc.csv', index_col=0)
    pc.index = pd.to_datetime(pc.index)

In [ ]:
1.3 Average Car Mileage
We want to know how annual mileage is changing in Wales. This was calculated using a query 'Annual Welsh Car Mileage by Fueltype', which is saved in BigQuery. That query essentially takes a selection of vehicles that have had an MOT in a Welsh postcode district, and then calculates its annual mileage, which is then allocated to a year (imperfectly). Lastly, we group them by a particular set of fueltypes (Petrol, Diesel and Electric, with hybrids grouped into Petrol) and the mean is calculated.

A note on this. Why do we take the mean? We want to know things about the total demand for vehicle miles (perhaps) more than the behaviour of particular outliers. Knowing the total number of vehicles and the average mileage gives us a good indication of this. It may limit our ability to consider how targeting high miler vehicles might accelerate the transition, but there will be some possibility to consider a reduction in the average mileage driven by fossil fuel. Also, the MOT database does not give us a complete picture of the whole vehicle parc, since it misses young vehicles. So using it a bit like a survey, taking the mean, and using that in conjunction with the total number of vehicles, seems like it could be a sensible approach.

A table in bigquery holds the data that we need, with the calculations already having being done for us. This table is rugged-baton-283921.wales.annual_mileage_fueltype. We can simply pull the data, and store in a csv.

In [5]:
%%bigquery mm --project rugged-baton-283921
SELECT *
FROM rugged-baton-283921.wales.annual_mileage_fueltype_cars;

Downloading: 100%|██████████| 72/72 [00:01<00:00, 56.77rows/s]
